In [1]:
import numpy as np
import random
import time
import re
import itertools
import scipy.spatial.distance

def load_model_vectors(model_path, words_set):
    vectors = {}
    with open(model_path, "r", encoding="utf8") as f:
        for line in f:
            tokens = line.rstrip().split(" ")
            word = tokens[0].lower()
            if word in words_set:
                vector = np.asarray(tokens[1:], dtype="float32")
                vectors[word] = vector
    return vectors

class Model:
    def __init__(self, model="glove.840B.300d.txt", dictionary="words.txt", pattern="^[a-z][a-z-]*[a-z]$"):
        words_set = set()
        with open(dictionary, "r", encoding="utf8") as f:
            for line in f:
                line = line.strip()
                if re.match(pattern, line):
                    words_set.add(line.lower())

        self.vectors_dict = load_model_vectors(model, words_set)
        self.words_array = np.array(list(self.vectors_dict.keys()))
        self.word_vectors = np.array(list(self.vectors_dict.values()))
        self.vectors = self.vectors_dict  # For compatibility

    def validate(self, word):
        """Clean up word and find best candidate to use"""
        # Strip unwanted characters
        clean = re.sub(r"[^a-zA-Z- ]+", "", word).strip().lower()
        if len(clean) <= 1:
            return None  # Word too short

        # Generate candidates for possible compound words
        candidates = []
        if " " in clean:
            candidates.append(re.sub(r" +", "-", clean))
            candidates.append(re.sub(r" +", "", clean))
        else:
            candidates.append(clean)
            if "-" in clean:
                candidates.append(re.sub(r"-+", "", clean))
        for cand in candidates:
            if cand in self.vectors:
                return cand  # Return first word that is in model
        return None  # Could not find valid word

    def distance(self, word1, word2):
        """Compute cosine distance (0 to 2) between two words"""
        vec1 = self.vectors.get(word1)
        vec2 = self.vectors.get(word2)
        if vec1 is not None and vec2 is not None:
            return scipy.spatial.distance.cosine(vec1, vec2)
        else:
            return 0.0  # Or handle as appropriate

    def dat(self, words, minimum=7):
        """Compute DAT score"""
        # Keep only valid unique words
        uniques = []
        for word in words:
            valid = self.validate(word)
            if valid and valid not in uniques:
                uniques.append(valid)

        # Keep subset of words
        if len(uniques) >= minimum:
            subset = uniques[:minimum]
        else:
            return None  # Not enough valid words

        # Compute distances between each pair of words
        distances = []
        for word1, word2 in itertools.combinations(subset, 2):
            dist = self.distance(word1, word2)
            distances.append(dist)

        # Compute the DAT score (average semantic distance multiplied by 100)
        return (sum(distances) / len(distances)) * 100

def find_max_dissimilar_words(model, k=7, num_iterations=5):
    # Initial greedy selection
    words = model.words_array
    word_vectors = model.word_vectors
    vectors = model.vectors

    selected_indices = []
    remaining_indices = np.arange(len(words))

    # Randomly select the first word
    first_idx = np.random.choice(remaining_indices)
    selected_indices.append(first_idx)
    remaining_indices = np.delete(remaining_indices, np.where(remaining_indices == first_idx))

    selected_vecs = [word_vectors[first_idx]]

    for _ in range(k - 1):
        # Compute distances from selected words to all remaining words
        dists = scipy.spatial.distance.cdist(word_vectors[remaining_indices], np.vstack(selected_vecs), 'cosine')
        min_dists = np.min(dists, axis=1)

        # Select the word with the maximum of these minimum distances
        max_min_dist_idx = np.argmax(min_dists)
        next_idx = remaining_indices[max_min_dist_idx]
        selected_indices.append(next_idx)
        selected_vecs.append(word_vectors[next_idx])

        # Remove selected index from remaining
        remaining_indices = np.delete(remaining_indices, max_min_dist_idx)

    selected_words = [words[idx] for idx in selected_indices]
    best_score = model.dat(selected_words)
    best_words = selected_words.copy()

    # Iterative refinement
    for iteration in range(num_iterations):
        # Compute average distance of each word to other words
        avg_distances = []
        for i in range(len(selected_words)):
            word_i = selected_words[i]
            distances = []
            for j in range(len(selected_words)):
                if i != j:
                    dist = model.distance(word_i, selected_words[j])
                    distances.append(dist)
            avg_distance = np.mean(distances)
            avg_distances.append(avg_distance)

        # Identify the word with the lowest average distance
        min_avg_dist_idx = np.argmin(avg_distances)
        word_to_replace = selected_words[min_avg_dist_idx]
        word_to_replace_idx = selected_indices[min_avg_dist_idx]

        # Remove this word from the set
        selected_words.pop(min_avg_dist_idx)
        selected_indices.pop(min_avg_dist_idx)
        selected_vecs.pop(min_avg_dist_idx)

        # Add the index back to remaining indices
        remaining_indices = np.append(remaining_indices, word_to_replace_idx)
        remaining_indices.sort()

        # Find a new word to replace it
        # Compute distances between the removed word and remaining words
        removed_vec = word_vectors[word_to_replace_idx].reshape(1, -1)
        dists_to_removed = scipy.spatial.distance.cdist(word_vectors[remaining_indices], removed_vec, 'cosine').flatten()

        # Also consider distances to current selected words
        if selected_vecs:
            dists_to_selected = scipy.spatial.distance.cdist(word_vectors[remaining_indices], np.vstack(selected_vecs), 'cosine')
            min_dists_to_selected = np.min(dists_to_selected, axis=1)
        else:
            # If no selected words (unlikely), set min distances to zeros
            min_dists_to_selected = np.zeros(len(remaining_indices))

        # Combine distances (we can adjust weights if desired)
        combined_scores = min_dists_to_selected + dists_to_removed

        # Select the word with the maximum combined score
        max_score_idx = np.argmax(combined_scores)
        new_idx = remaining_indices[max_score_idx]

        # Add the new word to the set
        selected_indices.append(new_idx)
        selected_words.append(words[new_idx])
        selected_vecs.append(word_vectors[new_idx])

        # Remove the new index from remaining indices
        remaining_indices = np.delete(remaining_indices, max_score_idx)

        # Recalculate the DAT score
        new_score = model.dat(selected_words)
        if new_score is not None and new_score > best_score:
            best_score = new_score
            best_words = selected_words.copy()
        else:
            # If no improvement, revert changes
            # Remove the added word and restore the removed word
            selected_indices.pop()
            selected_words.pop()
            selected_vecs.pop()
            remaining_indices = np.append(remaining_indices, new_idx)
            remaining_indices.sort()

            selected_indices.insert(min_avg_dist_idx, word_to_replace_idx)
            selected_words.insert(min_avg_dist_idx, word_to_replace)
            selected_vecs.insert(min_avg_dist_idx, word_vectors[word_to_replace_idx])
            remaining_indices = remaining_indices[remaining_indices != word_to_replace_idx]
            remaining_indices.sort()

    return best_words, best_score

In [2]:
model = Model(model="../../data/glove.840B.300d.txt", dictionary="../../data/words.txt")

In [13]:
i = 10

tries = []

for j in range(i):
    print(f"iteration: {j}")
    best_words, best_score = find_max_dissimilar_words(model, k=7, num_iterations=30)
    tries.append((best_words, best_score))

tries.sort(key=lambda tries: tries[1], reverse=True)

In [14]:
for best_words, best_score in tries:
    print("Words:", best_words)
    print("DAT Score:", best_score)

Words: ['tangency', 'dress', 'invention', 'interestingly', 'vac', 'tiff', 'dietitians']
DAT Score: 105.75741213702021
Words: ['plundering', 'rocky', 'interestingly', 'tiff', 'vac', 'sec', 'headrail']
DAT Score: 105.5264950885127
Words: ['his', 'raid', 'quote', 'antonyms', 'fir', 'xviii', 'hwy']
DAT Score: 105.19561279299003
Words: ['unspoken', 'gnome', 'sec', 'scuba', 'meted', 'coneflower', 'arm']
DAT Score: 105.01957884989679
Words: ['dandified', 'sec', 'blast', 'acer', 'forklifts', 'carisoprodol', 'culminated']
DAT Score: 104.9819339722013
Words: ['tails', 'batching', 'interestingly', 'sec', 'boulevards', 'arm', 'refoulement']
DAT Score: 104.50428908031124
Words: ['elysian', 'interestingly', 'sec', 'tachycardia', 'learn', 'cokes', 'acer']
DAT Score: 104.36084837613937
Words: ['patronisingly', 'date', 'jumper', 'acer', 'lactations', 'elf', 'plc']
DAT Score: 104.04705697770365
Words: ['timidity', 'jun', 'duty', 'arch-rivals', 'sac', 'analyzes', 'qwerty']
DAT Score: 103.52325921412557
W